# \[set-up\], imports
Reminder (if working on the notebook server): make sure your copy of `timing_analysis` is up to date and you're working on a development branch, e.g. `psr/J1234+5678/jks`. See README for more details. If not working on the notebook server, you may need to update required software packages (`pint`, `enterprise`, `enterprise_extensions`) according to requirements in `setup.py`.

In [ ]:
import timing_analysis.par_checker as pc
import timing_analysis.dmx_utils as du
import timing_analysis.lite_utils as lu
import timing_analysis.noise_utils as nu
import timing_analysis.plot_utils as pu
from timing_analysis.ftester import run_Ftests
from timing_analysis.utils import resid_stats, pdf_writer, check_recentness_excision, check_recentness_noise
from timing_analysis.timingconfiguration import TimingConfiguration
from astropy import log
from pint.fitter import ConvergenceFailure
import pint.fitter
from pint.utils import dmxparse
import os
import copy
from astropy.visualization import quantity_support
quantity_support()

# notebook gives interactive plots but not until the kernel is done
%matplotlib notebook
# inline gives non-interactive plots right away
#%matplotlib inline

In [ ]:
# default assumption is that notebook is being run by hand
autorun = False
run_Ftest = True  # Set to False if you don't need F-tests and want a faster notebook run!
check_excision = True
num_noise_iter = 2e5
use_dmx = False

if not autorun:
    run_noise_analysis = False 
    use_existing_noise_dir = True
    use_toa_pickle = True
else:
    run_noise_analysis = True 
    use_existing_noise_dir = False
    use_toa_pickle = False

In [ ]:
log.setLevel("INFO") # Set desired verbosity of log statements (DEBUG/INFO/WARNING/ERROR)
log_to_file = False

lu.git_config_info()

# develop/update \[prenoise\] timing solution

Load configuration (`.yaml`) file, get TOAs and timing model; if you're running from the root of the git distribution, simply edit the `.yaml` file name, otherwise include relevant paths to the `.yaml` file, and `.par`/`.tim` directories as kwargs (see commented example).

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)

using_wideband = tc.get_toa_type() == 'WB'

# Use excise.tim file if it exists, else revert to raw TOAs & initial cuts
mo,to = tc.get_model_and_toas(excised=False,usepickle=use_toa_pickle)
if not to: mo,to = tc.get_model_and_toas(apply_initial_cuts=False)
# Can write this back in when I finalize flag situation.
#tc.manual_cuts(to)

if log_to_file:
    lu.log_notebook_to_file(tc.get_source(), tc.get_toa_type())
    lu.log_warnings()

In [ ]:
# Computing pulse numbers ensures param changes in the model will not break phase connection
to.compute_pulse_numbers(mo)

In [ ]:
# Ensure DMX windows are calculated properly, set non-binary epochs to the center of the data span
if use_dmx == True:
    to = du.setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())
    lu.center_epochs(mo,to)
if not autorun: to.print_summary()

Run basic checks for pulsar name, solar system ephemeris, clock correction, ecliptic coordinates, tropospheric delays, planet Shapiro delays, and if applicable, removal of Arecibo data affected by bad LO. Check that TOAs being used are from the latest `toagen` release. Also check for the appropriate number of receiver JUMPs and DMJUMPs and fix them automatically if necessary.

In [ ]:
pc.check_settings(mo,to)

receivers = lu.get_receivers(to)
lu.add_feJumps(mo,receivers)
if using_wideband:
    lu.add_feDMJumps(mo,receivers)
pc.check_jumps(mo,receivers,toa_type=tc.get_toa_type())

if not autorun: check_recentness_excision(tc)

In [ ]:
# Define the fitter object and plot pre-fit residuals
fo = tc.construct_fitter(to,mo)
pu.plot_residuals_time(fo, restype='prefit')
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='prefit')
if using_wideband:
    pu.plot_dm_residuals(fo, restype='prefit')

In [ ]:
# Check that free-params follow NANOGrav conventions, fit
fo.model.free_params = tc.get_free_params(fo)
lu.check_fit(fo,skip_check=tc.skip_check)

try:
    fo.fit_toas(maxiter=tc.get_niter())
    fo.model.CHI2.value = fo.resids.chi2
except ConvergenceFailure:
    run_Ftest = False
    log.warning('Failed to converge; moving on with best result, but should address before final version.')

In [ ]:
# Plot post-fit residuals, print summary of results, write prenoise solution
pu.plot_residuals_time(fo, restype='postfit')
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='postfit')
if using_wideband:
    pu.plot_dm_residuals(fo, restype='postfit')
    
if not autorun: fo.print_summary()
lu.check_convergence(fo)

lu.write_par(fo,toatype=tc.get_toa_type(),addext='_prenoise')